## Business Problem
### Description
*** Ref: https://www.kaggle.com/c/quora-question-pairs/***

* Quora is a place to gain and share knowledge—about anything. It’s a platform to ask questions and connect with people who contribute unique insights and quality answers. This empowers people to learn from each other and to better understand the world.<br/> <br/>

* Over 100 million people visit Quora every month, so it's no surprise that many people ask similarly worded questions. Multiple questions with the same intent can cause seekers to spend more time finding the best answer to their question, and make writers feel they need to answer multiple versions of the same question. Quora values canonical questions because they provide a better experience to active seekers and writers, and offer more value to both of these groups in the long term.

### Problem Statement
* Identify which questions asked on Quora are duplicates of questions that have already been asked. This could be useful to instantly provide answers to questions that have already been answered. <br/>
** We are tasked with predicting whether a pair of questions are duplicates or not.**

### Identifying Business Constraints Problem Formulation
1. The cost of a mis-classification can be very high. i.e False Positives should be highly penalised, we do not want to link dissimilar questions together as the trust of user will reduce because of this.
2. We would want a probability of a pair of questions to be duplicates so that you can choose any threshold of choice.
3. No strict latency concerns.
4. Interpretability is partially important.
5. It is a binary classification problem, for a given pair of questions we need to predict if they are duplicate or not.

### Data overview
- Data will be in a file Train.csv <br>
- Train.csv contains 5 columns : qid1, qid2, question1, question2, is_duplicate <br>
- Size of Train.csv - 60MB <br>
- Number of rows in Train.csv = 404,290

### Performance Metric (decided by kaggle)
Source: https://www.kaggle.com/c/quora-question-pairs#evaluation

Metric(s): 
1. log-loss : https://www.kaggle.com/wiki/LogarithmicLoss
2. Binary Confusion Matrix

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from subprocess import check_output
%matplotlib inline
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import os
import gc
import os
import re
import pickle as pk
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS
import distance
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')
from IPython.display import display,Markdown
from tqdm import tqdm
from nltk.stem import WordNetLemmatizer 

os.chdir('E:/Projects/QuoraQuestionPairSimilarity')

In [ ]:
df = pd.read_csv("train.csv")
print("Number of data points:",df.shape[0])

In [ ]:
df.head()

In [ ]:
df.info()

### Observations:
We are given a minimal number of data fields here, consisting of:

- id:  Looks like a simple rowID
- qid{1, 2}:  The unique ID of each question in the pair
- question{1, 2}:  The actual textual contents of the questions.
- is_duplicate:  The label that we are trying to predict - whether the two questions are duplicates of each other.

### Distribution of data points among output classes
- Number of duplicate(smilar) and non-duplicate(non similar) questions

In [ ]:
df.groupby("is_duplicate")['id'].count().plot.bar()
plt.show()

In [ ]:
display(Markdown('''### Observations: 
* Total number of question pairs for training:\n   {}
* Question pairs are not Similar (is_duplicate = 0):\n   {}%
* Question pairs are Similar (is_duplicate = 1):\n   {}%'''.format(len(df),100 - round(df['is_duplicate'].mean()*100, 2),round(df['is_duplicate'].mean()*100, 2))))

### Number of unique questions

In [ ]:
qids = pd.Series(df['qid1'].tolist() + df['qid2'].tolist())
unique_qs = len(np.unique(qids))
qs_morethan_onetime = np.sum(qids.value_counts() > 1)

q_vals=qids.value_counts()

q_vals=q_vals.values

In [ ]:
#checking whether there are any repeated pair of questions
pair_duplicates = df[['qid1','qid2','is_duplicate']].groupby(['qid1','qid2']).count().reset_index()

In [ ]:
x = ["unique_questions" , "Repeated Questions"]
y =  [unique_qs , qs_morethan_onetime]

plt.figure(figsize=(10, 6))
plt.title ("Plot representing unique and repeated questions  ")
sns.barplot(x,y)
plt.show()

### Number of occurrences of each question

In [ ]:
plt.figure(figsize=(20, 10))

plt.hist(qids.value_counts(), bins=160)

plt.yscale('log', nonposy='clip')

plt.title('Log-Histogram of question appearance counts')

plt.xlabel('Number of occurences of question')

plt.ylabel('Number of questions')
plt.show()

### Checking for NULL values

In [ ]:
#Checking whether there are any rows with null values
nan_rows = df[df.isnull().any(1)]
print (nan_rows)

In [ ]:
# Filling the null values with ' '
df = df.fillna('')
nan_rows = df[df.isnull().any(1)]
print (nan_rows)

In [ ]:
display(Markdown('''### Observations:
* Total number of  Unique Questions are: {0}
* Number of unique questions that appear more than one time: {1} ({2}%)
* Max number of times a single question is repeated: {3}
* Number of duplicate questions {4}
* There are two rows with null values in question2 and one null value in question1 '''.format(unique_qs,qs_morethan_onetime,round(qs_morethan_onetime/unique_qs*100,3),max(qids.value_counts()),(pair_duplicates).shape[0] - df.shape[0])))

### Major Observation On qid's

### Ref: https://www.kaggle.com/ashhafez/temporal-pattern-in-train-response-rates

**If we make the assumption that there's an underlying temporal pattern to the data, and use the qid values as a proxy for it (higher qid value implies more recent question), then re-sorting the train set by increasing qid and plotting the sliding window of mean response rate should show us some pattern.**

In [ ]:
df["qmax"]      = df.apply( lambda row: max(row["qid1"], row["qid2"]), axis=1 )
df              = df.sort_values(by=["qmax"], ascending=True)
df["dupe_rate"] = df.is_duplicate.rolling(window=500, min_periods=500).mean()
df["timeline"]  = np.arange(df.shape[0]) / float(df.shape[0])

In [ ]:
df.plot(x="timeline", y="dupe_rate", kind="line")
plt.ylabel("dupe_rate")
plt.show()

### Observations:
- The above pattern, and the ~16.5% LB response rate reported by others, imply that the Public LB (and possibly Private LB) are potentially sourced from more recent data than the training set.
-  the decreasing average duplicate ratio(rolling mean) with increasing QID — Mostprobably an indication of Quora’s improving algorithm with time, thus reducing the number of duplicate questions with increasing ID

### Note: I came up with following features after a lot of reading of kaggle discussions and after lots of hit and trail.

### Basic Feature Extraction (before cleaning)

I used following features:
 - ____freq_qid1____ = Frequency of qid1's
 - ____freq_qid2____ = Frequency of qid2's 
 - ____q1len____ = Length of q1
 - ____q2len____ = Length of q2
 - ____q1_n_words____ = Number of words in Question 1
 - ____q2_n_words____ = Number of words in Question 2
 - ____word_Common____ = (Number of common unique words in Question 1 and Question 2)
 - ____word_Total____ =(Total num of words in Question 1 + Total num of words in Question 2)
 - ____word_share____ = (word_common)/(word_Total)
 - ____freq_q1+freq_q2____ = sum total of frequency of qid1 and qid2 
 - ____freq_q1-freq_q2____ = absolute difference of frequency of qid1 and qid2 

In [ ]:
# Ref: https://towardsdatascience.com/apply-function-to-pandas-dataframe-rows-76df74165ee4
# Ref: https://stackoverflow.com/questions/33518124/how-to-apply-a-function-on-every-row-on-a-dataframe
if os.path.isfile('df_fe_without_preprocessing_train.pkl'):
    df = pd.read_pickle("df_fe_without_preprocessing_train.pkl")
    print("Loaded file!!")
elif isinstance(df,pd.core.frame.DataFrame):
    df['freq_qid1'] = df.groupby('qid1')['qid1'].transform('count') 
    df['freq_qid2'] = df.groupby('qid2')['qid2'].transform('count')
    df['q1len'] = df['question1'].str.len() 
    df['q2len'] = df['question2'].str.len()
    df['q1_n_words'] = df['question1'].apply(lambda row: len(row.split(" ")))
    df['q2_n_words'] = df['question2'].apply(lambda row: len(row.split(" ")))

    def normalized_word_Common(row):
        w1 = set(map(lambda word: word.lower().strip(), row['question1'].split(" ")))
        w2 = set(map(lambda word: word.lower().strip(), row['question2'].split(" ")))    
        return 1.0 * len(w1 & w2)
    df['word_Common'] = df.apply(normalized_word_Common, axis=1)

    def normalized_word_Total(row):
        w1 = set(map(lambda word: word.lower().strip(), row['question1'].split(" ")))
        w2 = set(map(lambda word: word.lower().strip(), row['question2'].split(" ")))    
        return 1.0 * (len(w1) + len(w2))
    df['word_Total'] = df.apply(normalized_word_Total, axis=1)

    def normalized_word_share(row):
        w1 = set(map(lambda word: word.lower().strip(), row['question1'].split(" ")))
        w2 = set(map(lambda word: word.lower().strip(), row['question2'].split(" ")))    
        return 1.0 * len(w1 & w2)/(len(w1) + len(w2))
    df['word_share'] = df.apply(normalized_word_share, axis=1)

    df['freq_q1+q2'] = df['freq_qid1']+df['freq_qid2']
    df['freq_q1-q2'] = abs(df['freq_qid1']-df['freq_qid2'])

    df.to_pickle("df_fe_without_preprocessing_train.pkl")
else:
    print("Unexpected input encountered")

df.head()

In [ ]:
df_basic_features = df[["freq_qid1","freq_qid2","q1len","q2len","q1_n_words","q2_n_words","word_Common","word_Total","word_share","freq_q1+q2","freq_q1-q2"]]
df_basic_features.describe()

### Analysis of some of the extracted features

In [ ]:
display(Markdown('''### Observations:
1. Here some questions have only one single word and some also have one letter.
2. Minimum  and Maximum no.of words of the questions in question1 respectively are {} and {}
3. Minimum and Maximum no.of words of the questions in question2 respectively are {} and {}
4. Number of Questions with minimum length [question1] : {}
5. Number of Questions with minimum length [question2] : {}
6. The Above Values show high variance in the dataset.
7. I may be adding multicollinearity to the data which I've dealt before finalizing my train data.'''.format(min(df['q1_n_words']),max(df['q1_n_words']),min(df['q2_n_words']),max(df['q2_n_words']),df[df['q1_n_words']== 1].shape[0],df[df['q2_n_words']== 1].shape[0])))

In [ ]:
f= plt.figure(figsize=(12, 8))
f.suptitle('Feature: word_share')
plt.subplot(1,2,1)
sns.violinplot(x = 'is_duplicate', y = 'word_share', data = df[0:])
plt.subplot(1,2,2)
sns.distplot(df[df['is_duplicate'] == 1.0]['word_share'][0:] , label = "1", color = 'red')
sns.distplot(df[df['is_duplicate'] == 0.0]['word_share'][0:] , label = "0" , color = 'blue' )
plt.legend()
plt.show()

In [ ]:
f = plt.figure(figsize=(12, 8))
f.suptitle('Feature: word_Common')
plt.subplot(1,2,1)
sns.violinplot(x = 'is_duplicate', y = 'word_Common', data = df[0:])
plt.subplot(1,2,2)
sns.distplot(df[df['is_duplicate'] == 1.0]['word_Common'][0:] , label = "1", color = 'red')
sns.distplot(df[df['is_duplicate'] == 0.0]['word_Common'][0:] , label = "0" , color = 'blue' )
plt.legend()
plt.show()

### Observations
- The distributions for normalized word_share have some overlap on the far right-hand side, i.e., there are quite a lot of questions with high word similarity.
- The distributions for normalized word_share w.r.t hue=
- The average word share and Common no. of words of qid1 and qid2 is more when they are duplicate(Similar)
- The distributions of the word_Common feature in similar and non-similar questions are highly overlapping.Thus, this feature may give less feature importance.
- drop word_common as it has high overlap. Let's take some other transformation of in feature eng stage.

### Preprocessing of Text
** This Involves: **
- Removing html tags 
- Removing Punctuations
- Performing stemming
- Removing Stopwords
- Expanding contractions etc.

In [ ]:
# Reason Why I Used WordNet ref: https://www.kaggle.com/c/quora-question-pairs/discussion/30340
# import nltk
# nltk.download('wordnet')

In [ ]:
import joblib as jb

In [ ]:
# To get the results in 4 decemal points
# Ref: https://stackoverflow.com/questions/18603270/progress-indicator-during-pandas-operations
class Preprocessor:
    def __init__(self,lemma):
        self.lemma=lemma

    def preprocess(self,x):
        '''This fuction is used to clan data and replace text with some common conventions used in emperical use of language.'''
        x = str(x).lower()
        x = x.replace(",000,000", "m").replace(",000", "k").replace("′", "'").replace("’", "'")\
                               .replace("won't", "will not").replace("cannot", "can not").replace("can't", "can not")\
                               .replace("n't", " not").replace("what's", "what is").replace("it's", "it is")\
                               .replace("'ve", " have").replace("i'm", "i am").replace("'re", " are")\
                               .replace("he's", "he is").replace("she's", "she is").replace("'s", " own")\
                               .replace("%", " percent ").replace("₹", " rupee ").replace("$", " dollar ")\
                               .replace("€", " euro ").replace("'ll", " will")
        x = re.sub(r"([0-9]+)000000", r"\1m", x)
        x = re.sub(r"([0-9]+)000", r"\1k", x)


        if self.lemma:
            lemmatizer = WordNetLemmatizer()
        else:
            porter = PorterStemmer() 
        pattern = re.compile('\W')

        if isinstance(x,str):
            x = re.sub(pattern, ' ', x)    
        if isinstance(x,str):
            x = ' '.join(list(map(lambda y: lemmatizer.lemmatize(y),x.split(' ')))) if self.lemma else ' '.join(list(map(lambda y: porter.stem(y),x.split(' '))))
            example1 = BeautifulSoup(x)
            x = example1.get_text()


        return x
    
    def call_preprocess(self,data):
        tqdm.pandas()
        temp = 'lemma' if self.lemma else 'stem'
        try:
            data = pd.read_pickle("df_fe_"+temp+"_preprocessed_train.pkl")
            print("File loaded!!")
        except:
            data["question1"] = data["question1"].fillna("")
            data["question2"] = data["question2"].fillna("")
            data["question1"] = jb.Parallel(n_jobs=8)(jb.delayed(self.preprocess)(row) for row in tqdm(data["question1"]))
            data["question2"] = jb.Parallel(n_jobs=8)(jb.delayed(self.preprocess)(row) for row in tqdm(data["question2"]))
            data.to_pickle("df_fe_"+temp+"_preprocessed_train.pkl")
        finally:
            print("Done!")

In [ ]:
process = Preprocessor(True)
process.call_preprocess(df)

In [ ]:
df.head(3)

In [ ]:
df_basic_features = df[df_basic_features.columns]

### Analysis on text features

- Creating Word Cloud of Duplicates and Non-Duplicates Question pairs
- We can observe the most frequent occuring words

In [ ]:
df_duplicate = df[df['is_duplicate'] == 1]
dfp_nonduplicate = df[df['is_duplicate'] == 0]


p = np.dstack([df_duplicate["question1"], df_duplicate["question2"]]).flatten()
n = np.dstack([dfp_nonduplicate["question1"], dfp_nonduplicate["question2"]]).flatten()

print ("Number of data points in class 1 (duplicate pairs) :",len(p))
print ("Number of data points in class 0 (non duplicate pairs) :",len(n))

p=' '.join(list(map(lambda x:''.join(x),p)))
n=' '.join(list(map(lambda x:''.join(x),n)))

In [ ]:
# removing the Stop Words:
stopwords = set(STOPWORDS)
stopwords.add("said")
stopwords.add("br")
stopwords.add(" ")
stopwords.remove("not")

stopwords.remove("no")
#stopwords.remove("good")
#stopwords.remove("love")
# stopwords.remove("like")
#stopwords.remove("best")
#stopwords.remove("!")

#### Word Clouds generated from duplicate pair question's text

In [ ]:
wc = WordCloud(background_color="white", max_words=len(p), stopwords=stopwords)
wc.generate(p)
print ("Word Cloud for Duplicate Question pairs")
plt.imshow(wc, interpolation='bilinear')
plt.axis("off")
plt.show()

#### Word Clouds generated from non duplicate pair question's text

In [ ]:
wc = WordCloud(background_color="white", max_words=len(n),stopwords=stopwords)
# generate word cloud
wc.generate(n)
print ("Word Cloud for non-Duplicate Question pairs:")
plt.imshow(wc, interpolation='bilinear')
plt.axis("off")
plt.show()

### Observations:
-  Interesting aspect of the questions is that a disproportionate number seem to be from/about India. This may cause all of the NLP features to skew towards words relevant to India questions.
- I'm doubtful that the resulting features will generalize well to questions from/about different regions. Or we I may have to come up with more features to generalize this.

In [ ]:
list(enumerate(df_basic_features.columns))

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler =StandardScaler()
df_basic_features = scaler.fit_transform(df_basic_features)

In [ ]:
S = np.corrcoef(df_basic_features,rowvar=False)
sns.heatmap(S,annot=True,fmt='f',figure=plt.figure(figsize=(15,10)))
plt.show()

### Observations:
-  q1len and q1_n_words are very highly correlated and q2len and q2_n_words. I will drop these for now and add new features which are transformations of these features to deal with these.
- I'm doubtful that the resulting features will generalize well to questions from/about different regions. Or we I may have to come up with more features to generalize this.
- word_taotal has high correlation with  other features q1len, q2len, q1_n_words, q2_n_words.
-  freq_qid1 and freq_qid2 are very highly correlated with freq_q1+q2. 

In [ ]:
df = df.drop(["q1len","q1_n_words","q2len","q2_n_words","word_Common"],axis=1)

In [ ]:
try:
    df=pd.read_pickle("df_preprocessed.pkl")
    print("File Loaded!")
except:
    df.to_pickle("df_preprocessed.pkl")

In [ ]:
df.columns

In [ ]:
df.columns.shape